In [321]:
import torch
import numpy as np
import torch.nn as nn

In [118]:
#define the label function and loss function
def label(x1,x2):
    return (x1*x1+x2*x2)/2
def loss(predict,y):
    return sum((predict-y)**2)

In [133]:
#randomly select 100 tuple with 2 numbers between 0 and 1
size=100
node=torch.Tensor(size,2).uniform_(0,1)

In [134]:
#generate label 
y_label=label(node[:,0],node[:,1])

In [164]:
#define nerual network
class nn_model(nn.Module):
    def __init__(self,n_node):
        super(nn_model,self).__init__()
        self.linear1=nn.Linear(n_node,10)
        self.linear2=nn.Linear(10,10)
        self.linear3=nn.Linear(10,1)
        self.activation=nn.Sigmoid()
    def forward(self,inputs):
        y=self.linear1(inputs)
        y=self.activation(y)
        y=self.linear2(y)
        y=self.activation(y)
        y=self.linear3(y)
        return y
model=nn_model(2)
print(model)

nn_model(
  (linear1): Linear(in_features=2, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=1, bias=True)
  (activation): Sigmoid()
)


In [136]:
w1=model.linear1.weight
b1=model.linear1.bias
w2=model.linear2.weight
b2=model.linear2.bias
w3=model.linear3.weight
b3=model.linear3.bias

Parameter containing:
tensor([0.1149], requires_grad=True)

In [137]:
w1.requires_grad_(True)
b1.requires_grad_(True)
w2.requires_grad_(True)
b2.requires_grad_(True)
w3.requires_grad_(True)
b3.requires_grad_(True)

Parameter containing:
tensor([0.1149], requires_grad=True)

In [138]:
#define data iteration methods
def data_iter(batch_size,node, labels):
    num_examples = len(node)
    indices = list(range(num_examples))
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i:min(i + batch_size, num_examples)])
        yield  node.index_select(0, j),labels.index_select(0, j)

In [139]:
#use batch size ==1 and generate w.grad and b.grad in each step
epoch=1
batch_size=1
result=[]
weight_find=[]
i=0
for train_x,train_y in data_iter(batch_size,node,y_label):
    pred_y=model(train_x)
    l=loss(pred_y,train_y)
    model.zero_grad()
    l.backward()
#     ww1=torch.Tensor(w1)
    result.append((model.linear1.weight.grad.clone(),model.linear1.bias.grad.clone(),
                   model.linear2.weight.grad.clone(),model.linear2.bias.grad.clone(),
                   model.linear3.weight.grad.clone(),model.linear3.bias.grad.clone()))

In [140]:
##self implementation
w1.requires_grad_(False)
b1.requires_grad_(False)
w2.requires_grad_(False)
b2.requires_grad_(False)
w3.requires_grad_(False)
b3.requires_grad_(False)

def sigmoid(z):
    return 1.0/(1+np.exp(-z))
#forward propagation:
def forward_propagation(w1,b1,w2,b2,w3,b3,train_x):
    z1=train_x@(w1.T)+b1
#     print(z1)
    y1=sigmoid(z1)
#     print(y1)
    z2=y1@(w2.T)+b2
    y2=sigmoid(z2)
#     print(z2)
    z3=y2@(w3.T)+b3
#     print(z3)
    y3=z3
    return z1,y1,z2,y2,y3

def backward_propagation(y3,y2,z2,y1,z1,y_label,train_x,indicator):
    if indicator=="w1":
        return 2*sum(y3-y_label)*((w3*sigmoid(z2)*(1-sigmoid(z2)))@w2*sigmoid(z1)*(1-sigmoid(z1))).T@train_x
    elif indicator=='b1':
        return 2*sum(y3-y_label)*(w3*sigmoid(z2)*(1-sigmoid(z2)))@w2*sigmoid(z1)*(1-sigmoid(z1))
    elif indicator=='w2':
        return 2*sum(y3-y_label)*(w3*sigmoid(z2)*(1-sigmoid(z2))).T@y1
    elif indicator=='b2':
        return 2*sum(y3-y_label)*w3*sigmoid(z2)*(1-sigmoid(z2))
    elif indicator=='w3':
        return 2*sum(y3-y_label)*y2
    elif indicator=='b3':
        return 2*sum(y3-y_label)
    else:
        print("import indicator error")

In [158]:
result2=[]
i=0
for train_x,train_y in data_iter(batch_size,node,y_label):
    z1,y1,z2,y2,y3=forward_propagation(w1,b1,w2,b2,w3,b3,train_x)
    dl_dw3=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"w3")
    dl_db3=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"b3")
    dl_dw2=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"w2")
    dl_db2=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"b2")
    dl_dw1=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"w1")
    dl_db1=backward_propagation(y3,y2,z2,y1,z1,train_y,train_x,"b1")
    result2.append((dl_dw1,dl_db1[0],dl_dw2,dl_db2[0],dl_dw3,dl_db3))

In [159]:
def save_file(List_file,path):
    if isinstance(List_file,list):
        with open(path,"w") as f:
            for i in range(len(List_file)):
                f.write(str(i)+":"+"\n"+"W1_grad: "+str(List_file[i][0])+"\n"
                        +"b1_grad: "+str(List_file[i][1])+"\n"
                        +"W2_grad: "+str(List_file[i][2])+"\n"
                        +"b2_grad: "+str(List_file[i][3])+'\n'
                        +"W3_grad: "+str(List_file[i][4])+"\n"
                        +"b3_grad: "+str(List_file[i][5])+'\n'+"\n")

In [160]:
save_file(result,"./torch_autograd.dat")

In [161]:
save_file(result2,"./my_autograd.dat")